<a href="https://colab.research.google.com/github/JiHyun13/coding-academy-location-analysis/blob/main/%EC%BD%94%EB%94%A9_%ED%95%99%EC%9B%90_%EC%9E%85%EC%A7%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### 라이브러리

In [ ]:
# folium 설치 (처음 1회만)
!pip install folium
!pip install geopy
!pip install openpyxl
!pip install haversine

In [ ]:
import pandas as pd
import folium
from folium import plugins
import numpy as np
from geopy.distance import geodesic
import matplotlib.pyplot as plt
from collections import Counter
import seaborn as sns
from haversine import haversine, Unit

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### 1. map class 정의   

In [ ]:
class CompetitorAnalysisMapper:
    def __init__(self):
        self.competitor_keywords = [
            '코딩', '컴퓨터', '프로그래밍', '소프트웨어', 'SW', '인공지능','로봇', '게임', '앱개발', '웹개발', 'IT'
        ]

        self.supporting_keywords = [
            '논술', '독서', '토론', '음악', '태권도', '체육관', '바둑', '산수', '주산'
        ]

    def load_data(self, school_path, academy_path):
        """데이터 로드"""
        self.schools_df = pd.read_csv(school_path, encoding='euc-kr')
        self.academies_df = pd.read_csv(academy_path, encoding='utf-8')
        print(f"학교 데이터: {len(self.schools_df)}개")
        print(f"학원 데이터: {len(self.academies_df)}개")

    def filter_by_region(self, region='서울특별시'):
        """지역별 필터링"""
        # 학교 필터링 (초중학교만)
        school_types = ['초등학교', '중학교']
        self.schools = self.schools_df[
            (self.schools_df['시도교육청명'].str.contains(region, na=False)) &
            (self.schools_df['학교급구분'].isin(school_types)) &
            (self.schools_df['운영상태'] == '운영') &
            (self.schools_df['위도'].notna()) &
            (self.schools_df['경도'].notna()) &
            (self.schools_df['학생수'].notna())
        ].copy()

        # 학원 필터링 (개원 상태, 좌표 존재)
        self.academies = self.academies_df[
            (self.academies_df['위도'].notna()) &
            (self.academies_df['경도'].notna())
        ].copy()

        print(f"{region} 초중학교: {len(self.schools)}개")
        print(f"{region} 학원: {len(self.academies)}개")

    def classify_supporting_academy(self, academy):
        """상생업체 세부 분류"""
        text = ' '.join([
            str(academy.get('학원명', '')),
            str(academy.get('교습과정명', '')),
            str(academy.get('분야명', ''))
        ]).lower()

        if '음악' in text or '피아노' in text or '바이올린' in text:
            return '음악'
        elif '태권도' in text or '체육관' in text:
            return '태권도/체육관'
        elif '논술' in text or '독서' in text:
            return '논술/독서'
        elif '바둑' in text or '산수' in text or '주산' in text:
            return '바둑/주산'
        else:
            return '기타'  # None 대신 '기타' 반환

    def classify_academies(self):
        """학원 분류: 경쟁업체 vs 상생업체 (IT, AI 부분매칭 문제 해결)"""
        import re

        def is_exact_keyword_match(text, keyword):
            """정확한 단어 매칭 (부분 문자열 방지)"""
            pattern = r'\b' + re.escape(keyword.lower()) + r'\b'
            return bool(re.search(pattern, text.lower()))

        def get_academy_type(row):
            search_text = ' '.join([
                str(row.get('학원명', '')),
                str(row.get('교습과정명', '')),
                str(row.get('분야명', '')),
                str(row.get('교습계열명', ''))
            ])

            # 상생업체 먼저 체크 (우선순위)
            for keyword in self.supporting_keywords:
                if keyword.lower() in search_text.lower():
                    return 'supporting'

            # 경쟁업체 체크 (문제 키워드는 정확한 매칭)
            for keyword in self.competitor_keywords:
                if keyword.lower() in ['it', 'ai','메이커']:
                    # IT, AI는 단어 경계로 정확히 매칭
                    if is_exact_keyword_match(search_text, keyword):
                        return 'competitor'
                else:
                    # 나머지는 기존 방식 (부분 포함)
                    if keyword.lower() in search_text.lower():
                        return 'competitor'

            return 'other'

        self.academies['academy_type'] = self.academies.apply(get_academy_type, axis=1)

        # 분류 결과 및 좌표 정보 저장
        self.competitors = self.academies[self.academies['academy_type'] == 'competitor'].copy()
        self.supporting_academies = self.academies[self.academies['academy_type'] == 'supporting'].copy()

        # 좌표 컬럼명 통일 (x, y -> lat, lon)
        self.competitors['lat'] = self.competitors['위도']
        self.competitors['lon'] = self.competitors['경도']
        self.competitors['district'] = self.competitors['행정구역명']
        self.supporting_academies['lat'] = self.supporting_academies['위도']
        self.supporting_academies['lon'] = self.supporting_academies['경도']
        self.supporting_academies['district'] = self.supporting_academies['행정구역명']

        print(f"경쟁업체: {len(self.competitors)}개")
        print(f"상생업체: {len(self.supporting_academies)}개")
        print(f"기타: {len(self.academies[self.academies['academy_type'] == 'other'])}개")

        # 분류 확인용 출력
        print("\n🔴 경쟁업체 샘플:")
        for _, comp in self.competitors.head(3).iterrows():
            print(f"   - {comp['학원명']} ({comp.get('교습과정명', 'N/A')})")

        print("\n🟢 상생업체 샘플:")
        for _, sup in self.supporting_academies.head(3).iterrows():
            print(f"   - {sup['학원명']} ({sup.get('교습과정명', 'N/A')})")

    def analyze_competitor_surroundings(self, radius_km=1.0):
        """경쟁업체 주변 환경 분석"""
        surroundings_analysis = []

        for idx, competitor in self.competitors.iterrows():
            comp_lat, comp_lon = competitor['lat'], competitor['lon']

            # 반경 내 학교 찾기
            nearby_schools = []
            for _, school in self.schools.iterrows():
                distance = haversine((comp_lat, comp_lon), (school['위도'], school['경도']),unit=Unit.KILOMETERS)
                if distance <= radius_km:
                    nearby_schools.append({
                        'name': school['학교명'],
                        'type': school['학교급구분'],
                        'count': school['학생수'],
                        'distance': distance
                    })

            # 반경 내 상생업체 찾기
            nearby_supporting = []
            for _, academy in self.supporting_academies.iterrows():
                distance = haversine((comp_lat, comp_lon), (academy['lat'], academy['lon']),unit=Unit.KILOMETERS)
                if distance <= radius_km:
                    # 상생업체 타입 분류
                    academy_type = self.classify_supporting_academy(academy)
                    nearby_supporting.append({
                        'name': academy['학원명'],
                        'type': academy_type,
                        'distance': distance
                    })

            # 분석 결과 저장
            analysis = {
                'competitor_name': competitor['학원명'],
                'district': competitor.get('도로명주소', '').split()[1] if len(str(competitor.get('도로명주소', '')).split()) > 1 else 'Unknown',
                'lat': comp_lat,
                'lon': comp_lon,
                'nearby_elementary': len([s for s in nearby_schools if s['type'] == '초등학교']),
                'nearby_middle': len([s for s in nearby_schools if s['type'] == '중학교']),
                'nearby_elem_students': sum(s['count'] for s in nearby_schools if s['type'] == '초등학교'),
                'nearby_mid_students': sum(s['count'] for s in nearby_schools if s['type'] == '중학교'),
                'nearby_music': len([s for s in nearby_supporting if '음악' in s['type']]),
                'nearby_taekwondo': len([s for s in nearby_supporting if '태권도/체육관' in s['type']]),
                'nearby_essay': len([s for s in nearby_supporting if '논술' in s['type']]),
                'nearby_go': len([s for s in nearby_supporting if '바둑/주산' in s['type']]),
                'total_schools': len(nearby_schools),
                'total_supporting': len(nearby_supporting),
                'school_details': nearby_schools,
                'supporting_details': nearby_supporting
            }

            surroundings_analysis.append(analysis)

        self.surroundings_df = pd.DataFrame(surroundings_analysis)
        self.surroundings_df.to_csv('save1_surroundings_df.csv', index=False, encoding="utf-8-sig")
        return self.surroundings_df

    def generate_environment_summary(self):
        """환경 분석 요약"""
        if not hasattr(self, 'surroundings_df'):
            print("먼저 analyze_competitor_surroundings()를 실행하세요.")
            return

        df = self.surroundings_df

        print("="*50)
        print("🏢 경쟁업체 주변 환경 분석 결과")
        print("="*50)

        print(f"\n📍 분석 대상: {len(df)}개 경쟁업체 (반경 1km 기준)")

        print(f"\n🏫 주변 학교 현황:")
        print(f"   • 평균 초등학교: {df['nearby_elementary'].mean():.1f}개, 학생 수 {df['nearby_elem_students'].mean()}명")
        print(f"   • 평균 중학교: {df['nearby_middle'].mean():.1f}개, 학생 수 {df['nearby_mid_students'].mean()}명")
        print(f"   • 평균 총 학교 수: {df['total_schools'].mean():.1f}개")

        print(f"\n🎓 주변 상생업체 현황:")
        print(f"   • 평균 음악학원: {df['nearby_music'].mean():.1f}개")
        print(f"   • 평균 태권도학원: {df['nearby_taekwondo'].mean():.1f}개")
        print(f"   • 평균 논술학원: {df['nearby_essay'].mean():.1f}개")
        print(f"   • 평균 바둑/주산학원: {df['nearby_go'].mean():.1f}개")
        print(f"   • 평균 총 상생업체: {df['total_supporting'].mean():.1f}개")

        # 최적 환경 기준 설정
        optimal_conditions = {
            'min_elementary': df['nearby_elementary'].quantile(0.5),
            'min_middle': df['nearby_middle'].quantile(0.5),
            'min_elem_students': df['nearby_elem_students'].quantile(0.5),
            'min_mid_students': df['nearby_mid_students'].quantile(0.5),
            'min_supporting': df['total_supporting'].quantile(0.5)
        }

        print(f"\n✅ 최적 입지 조건 (50%):")
        print(f"   • 초등학교 {optimal_conditions['min_elementary']:.0f}개 이상, 초등학생 수 {optimal_conditions['min_elem_students']:.0f}명 이상")
        print(f"   • 중학교 {optimal_conditions['min_middle']:.0f}개 이상, 중학생 수 {optimal_conditions['min_mid_students']:.0f}명 이상")
        print(f"   • 상생업체 {optimal_conditions['min_supporting']:.0f}개 이상")

        return optimal_conditions

    def find_opportunity_locations(self, optimal_conditions):
        """입지 기회 발굴"""
        # 학교 주변 잠재 지역 찾기
        opportunity_locations = []

        # 각 학교를 중심으로 반경 1km 분석
        for _, school in self.schools.iterrows():
            school_lat, school_lon = school['위도'], school['경도']

            # 반경 내 경쟁업체 체크
            nearby_competitors = 0
            for _, comp in self.competitors.iterrows():
                distance = haversine((school_lat, school_lon), (comp['lat'], comp['lon']),unit=Unit.KILOMETERS)
                if distance <= 1.0:
                    nearby_competitors += 1

            # 경쟁업체가 없거나 적은 곳만 선별
            if nearby_competitors <= 1:  # 1개 이하
                # 주변 환경 분석
                nearby_schools = []
                nearby_supporting = []
                nearby_elem_students=[]
                nearby_mid_students=[]

                for _, other_school in self.schools.iterrows():
                    distance = haversine((school_lat, school_lon), (other_school['위도'], other_school['경도']),unit=Unit.KILOMETERS)
                    if distance <= 1.0:
                        nearby_schools.append(other_school['학교급구분'])
                        if other_school['학교급구분']=='초등학교':
                          nearby_elem_students.append(other_school['학생수'])
                        else:
                          nearby_mid_students.append(other_school['학생수'])

                for _, academy in self.supporting_academies.iterrows():
                    distance = haversine((school_lat, school_lon), (academy['lat'], academy['lon']),unit=Unit.KILOMETERS)
                    if distance <= 1.0:
                        academy_type = self.classify_supporting_academy(academy)
                        nearby_supporting.append(academy_type)

                # 최적 조건 충족하는지 확인
                elementary_count = nearby_schools.count('초등학교')
                middle_count = nearby_schools.count('중학교')
                elementary_students_count = sum(nearby_elem_students)
                middle_students_count = sum(nearby_mid_students)
                supporting_count = len(nearby_supporting)

                if (elementary_count >= optimal_conditions['min_elementary'] and
                    middle_count >= optimal_conditions['min_middle'] and
                    supporting_count >= optimal_conditions['min_supporting'] and
                    elementary_students_count >= optimal_conditions['min_elem_students'] and
                    middle_students_count >= optimal_conditions['min_mid_students']
                    ):

                    opportunity_locations.append({
                        'center_school': school['학교명'],
                        'center_school_type': school['학교급구분'],
                        'district': school['소재지도로명주소'].split()[1] if len(school['소재지도로명주소'].split()) > 1 else 'Unknown',
                        'lat': school_lat,
                        'lon': school_lon,
                        'nearby_elementary': elementary_count,
                        'nearby_middle': middle_count,
                        'nearby_elem_students': elementary_students_count,
                        'nearby_mid_students': middle_students_count,
                        'nearby_supporting': supporting_count,
                        'nearby_competitors': nearby_competitors,
                        'supporting_types': Counter(nearby_supporting)
                    })

        self.opportunities_df = pd.DataFrame(opportunity_locations)

        print(f"\n🎯 발굴된 입지 기회: {len(self.opportunities_df)}개 지역")

        if len(self.opportunities_df) > 0:
            print(f"\n📊 기회 지역 분포:")
            district_counts = self.opportunities_df['district'].value_counts().head(10)
            for district, count in district_counts.items():
                print(f"   • {district}: {count}개")

        return self.opportunities_df

    def create_comprehensive_map(self):
        """종합 분석 지도 생성"""
        # 서울 중심 지도
        m = folium.Map(
            location=[37.5665, 126.9780],
            zoom_start=11,
            tiles='CartoDB positron'
        )

        # 1. 학교 레이어
        school_group = folium.FeatureGroup(name="🏫 초중학교", show=True)
        for _, school in self.schools.iterrows():
            color = '#4285F4' if school['학교급구분'] == '초등학교' else '#34A853'
            folium.CircleMarker(
                location=[school['위도'], school['경도']],
                radius=3,
                popup=f"🏫 {school['학교명']}<br>{school['학교급구분']}",
                color=color,
                fill=True,
                fillOpacity=0.6,
                weight=1
            ).add_to(school_group)

        # 2. 경쟁업체 레이어
        competitor_group = folium.FeatureGroup(name="🔴 경쟁업체 (코딩학원)", show=True)
        for _, comp in self.competitors.iterrows():
            district = comp.get('도로명주소', '').split()[1] if len(str(comp.get('도로명주소', '')).split()) > 1 else 'Unknown'
            popup_content = f"""
            <div style="width: 200px;">
                <h4 style="color: red;">🔴 경쟁업체</h4>
                <b>{comp['학원명']}</b><br>
                과정: {comp.get('교습과정명', 'N/A')}<br>
                지역: {district}
            </div>
            """
            folium.Marker(
                location=[comp['lat'], comp['lon']],
                popup=folium.Popup(popup_content, max_width=250),
                icon=folium.Icon(color='red', icon='laptop', prefix='fa'),
                tooltip=comp['학원명']
            ).add_to(competitor_group)

        # 3. 상생업체 레이어 (전체)
        supporting_group = folium.FeatureGroup(name=f"🟢 상생업체 ({len(self.supporting_academies)}개)", show=False)
        for _, academy in self.supporting_academies.iterrows():
            academy_type = self.classify_supporting_academy(academy)
            district = academy.get('도로명주소', '').split()[1] if len(str(academy.get('도로명주소', '')).split()) > 1 else 'Unknown'
            color_map = {
                '음악': 'purple',
                '태권도/체육관': 'orange',
                '논술/독서': 'red',
                '바둑/주산': 'pink',
                '기타': 'gray'
            }

            folium.CircleMarker(
                location=[academy['lat'], academy['lon']],
                radius=1.5,  # 조금 더 작게
                popup=f"🟢 {academy['학원명']}<br>{academy_type}<br>{district}",
                color=color_map.get(academy_type, 'gray'),
                fill=True,
                fillOpacity=0.6,
                weight=0.5
            ).add_to(supporting_group)

        # 4. 기회 지역 레이어
        if hasattr(self, 'opportunities_df') and len(self.opportunities_df) > 0:
            opportunity_group = folium.FeatureGroup(name="⭐ 입지 기회", show=True)

            for _, opp in self.opportunities_df.iterrows():
                popup_content = f"""
                <div style="width: 250px;">
                    <h4 style="color: gold;">⭐ 입지 기회</h4>
                    <b>중심: {opp['center_school']}</b><br>
                    지역: {opp['district']}<br>
                    초등학교: {opp['nearby_elementary']}개<br>
                    초등학생: {opp['nearby_elem_students']}명<br>
                    중학교: {opp['nearby_middle']}개<br>
                    중학생: {opp['nearby_mid_students']}명<br>
                    상생업체: {opp['nearby_supporting']}개<br>
                    경쟁업체: {opp['nearby_competitors']}개
                </div>
                """

                folium.Marker(
                    location=[opp['lat'], opp['lon']],
                    popup=folium.Popup(popup_content, max_width=300),
                    icon=folium.Icon(color='lightgreen', icon='star', prefix='fa'),
                    tooltip=f"기회 지역: {opp['district']}"
                ).add_to(opportunity_group)

            opportunity_group.add_to(m)

        # 레이어 추가
        school_group.add_to(m)
        competitor_group.add_to(m)
        supporting_group.add_to(m)

        # 레이어 컨트롤
        folium.LayerControl(position='topright', collapsed=False).add_to(m)

        # 범례
        legend_html = '''
        <div style="position: fixed; bottom: 50px; left: 50px; width: 280px;
                    background-color: white; border:2px solid grey; z-index:9999;
                    font-size:12px; padding: 10px;">
        <h4 style="margin-top:0;">경쟁 분석 지도</h4>
        <p><i class="fa fa-circle" style="color:#4285F4"></i> 초등학교</p>
        <p><i class="fa fa-circle" style="color:#34A853"></i> 중학교</p>
        <p><i class="fa fa-map-marker" style="color:red"></i> 경쟁업체 (코딩학원)</p>
        <p><i class="fa fa-circle" style="color:blue"></i> 상생업체 (음악/바둑 등)</p>
        <p><i class="fa fa-star" style="color:lightgreen"></i> 입지 기회 지역</p>
        </div>
        '''
        m.get_root().html.add_child(folium.Element(legend_html))

        return m

#### 2. 실행 함수

In [ ]:
# 실행 함수들
def run_complete_analysis():
    """전체 분석 실행"""
    analyzer = CompetitorAnalysisMapper()

    # 1. 데이터 로드
    school_path = '/content/drive/MyDrive/코딩학원/서울학교데이터_학생수.csv'
    academy_path = '/content/drive/MyDrive/코딩학원/최종학원.csv'
    analyzer.load_data(school_path, academy_path)

    # 2. 지역 필터링
    analyzer.filter_by_region('서울특별시')

    # 3. 학원 분류, 4삭제
    analyzer.classify_academies()

    # 5. 경쟁업체 주변 환경 분석
    print("\n🔍 경쟁업체 주변 환경 분석 중...")
    analyzer.analyze_competitor_surroundings(radius_km=1.0)

    # 6. 환경 분석 요약
    optimal_conditions = analyzer.generate_environment_summary()

    # 7. 입지 기회 발굴
    print("\n🎯 입지 기회 발굴 중...")
    analyzer.find_opportunity_locations(optimal_conditions)

    # 8. 종합 지도 생성
    print("\n🗺️ 종합 분석 지도 생성 중...")
    map_obj = analyzer.create_comprehensive_map()
    map_obj.save('competitor_analysis_map.html')

    print("\n✅ 분석 완료!")
    print("📁 파일 저장: competitor_analysis_map.html")

    return analyzer

#### 3. 보고서 생성 및 저장 함수

In [ ]:
def generate_detailed_report(analyzer, save_files=True):
    """상세 분석 보고서 생성 및 저장"""
    if not hasattr(analyzer, 'surroundings_df'):
        print("먼저 분석을 실행하세요.")
        return

    # 현재 시간으로 파일명 생성
    from datetime import datetime
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    # 보고서 텍스트 생성
    report_content = []

    report_content.append("="*60)
    report_content.append("📊 코딩학원 경쟁 분석 보고서")
    report_content.append("="*60)
    report_content.append(f"📅 생성일시: {datetime.now().strftime('%Y년 %m월 %d일 %H시 %M분')}")
    report_content.append("")

    # 1. 경쟁업체 현황
    report_content.append("1️⃣ 경쟁업체 현황")
    report_content.append(f"   총 {len(analyzer.competitors)}개 경쟁업체 확인")
    report_content.append("")

    # 구별 분포
    district_dist = analyzer.competitors['district'].value_counts().head(10)
    report_content.append("   📍 구별 경쟁업체 분포 (상위 10개):")
    for district, count in district_dist.items():
        report_content.append(f"      {district}: {count}개")
    report_content.append("")

    # 2. 평균 환경 분석
    df = analyzer.surroundings_df
    report_content.append("2️⃣ 경쟁업체 평균 입지 환경 (반경 1km 기준)")
    report_content.append(f"   🏫 주변 학교:")
    report_content.append(f"      - 평균 초등학교: {df['nearby_elementary'].mean():.1f}개")
    report_content.append(f"      - 평균 중학교: {df['nearby_middle'].mean():.1f}개")
    report_content.append(f"      - 평균 총 학교: {df['total_schools'].mean():.1f}개")
    report_content.append("")

    report_content.append(f"   🎓 주변 상생업체:")
    report_content.append(f"      - 평균 음악학원: {df['nearby_music'].mean():.1f}개")
    report_content.append(f"      - 평균 태권도학원: {df['nearby_taekwondo'].mean():.1f}개")
    report_content.append(f"      - 평균 논술학원: {df['nearby_essay'].mean():.1f}개")
    report_content.append(f"      - 평균 바둑학원: {df['nearby_go'].mean():.1f}개")
    report_content.append(f"      - 평균 총 상생업체: {df['total_supporting'].mean():.1f}개")
    report_content.append("")

    # 3. 최적 입지 조건
    optimal_conditions = {
        'min_elementary': df['nearby_elementary'].quantile(0.50),
        'min_middle': df['nearby_middle'].quantile(0.50),
        'min_supporting': df['total_supporting'].quantile(0.50),
        'min_elem_students': df['nearby_elem_students'].quantile(0.50),
        'min_mid_students': df['nearby_mid_students'].quantile(0.50)
    }

    report_content.append("   ✅ 최적 입지 조건 (경쟁업체 50% 수준):")
    report_content.append(f"      - 초등학교 {optimal_conditions['min_elementary']:.0f}개 이상, 학생 {optimal_conditions['min_elem_students']} 명 이상")
    report_content.append(f"      - 중학교 {optimal_conditions['min_middle']:.0f}개 이상,  학생 {optimal_conditions['min_mid_students']} 명 이상")
    report_content.append(f"      - 상생업체 {optimal_conditions['min_supporting']:.0f}개 이상")
    report_content.append("")

    # 4. 입지 기회
    if hasattr(analyzer, 'opportunities_df'):
        report_content.append("3️⃣ 입지 기회 지역")
        report_content.append(f"   🎯 총 {len(analyzer.opportunities_df)}개 기회 지역 발굴")
        report_content.append("      (최적 조건을 만족하면서 경쟁업체가 1개 이하인 지역)")
        report_content.append("")

        if len(analyzer.opportunities_df) > 0:
            opp_districts = analyzer.opportunities_df['district'].value_counts().head(10)
            report_content.append("   📍 구별 기회 지역 분포:")
            for district, count in opp_districts.items():
                report_content.append(f"      {district}: {count}개")
            report_content.append("")

            # 상위 기회 지역 상세 정보
            top_opportunities = analyzer.opportunities_df.nlargest(10, 'nearby_supporting')
            report_content.append("   🌟 추천 입지 순위 (상생업체 수 기준 상위 10개):")
            for idx, (_, opp) in enumerate(top_opportunities.iterrows(), 1):
                report_content.append(f"      {idx}. {opp['center_school']} 주변 ({opp['district']})")
                report_content.append(f"         초등학교: {opp['nearby_elementary']}개, 중학교: {opp['nearby_middle']}개")
                report_content.append(f"         상생업체: {opp['nearby_supporting']}개, 경쟁업체: {opp['nearby_competitors']}개")
                report_content.append("")

    # 5. 종합 결론
    report_content.append("4️⃣ 종합 결론 및 권장사항")
    report_content.append("   📈 시장 현황:")
    report_content.append(f"      - 서울 지역에 총 {len(analyzer.competitors)}개 코딩학원 경쟁업체 확인")
    report_content.append(f"      - 경쟁이 가장 치열한 지역: {district_dist.index[0]} ({district_dist.iloc[0]}개)")
    report_content.append("")

    avg_elementary = df['nearby_elementary'].mean()
    avg_supporting = df['total_supporting'].mean()

    report_content.append("   🎯 권장 입지 전략:")
    report_content.append(f"      - 초등학교 {avg_elementary:.0f}개 이상 위치한 지역 선호")
    report_content.append(f"      - 상생업체 {avg_supporting:.0f}개 이상 밀집된 교육 환경")
    report_content.append("      - 기존 경쟁업체가 없거나 1개 이하인 지역")

    if hasattr(analyzer, 'opportunities_df') and len(analyzer.opportunities_df) > 0:
        best_district = analyzer.opportunities_df['district'].value_counts().index[0]
        report_content.append(f"      - 최우선 검토 지역: {best_district}")

    # 콘솔에 출력
    print("\n".join(report_content))

    # 파일 저장
    if save_files:
        # 1. 텍스트 보고서 저장
        txt_filename = f'경쟁분석보고서_{timestamp}.txt'
        with open(txt_filename, 'w', encoding='utf-8') as f:
            f.write("\n".join(report_content))

        # 2. CSV 데이터 저장
        csv_filename = f'경쟁업체데이터_{timestamp}.csv'
        analyzer.surroundings_df.to_csv(csv_filename, encoding='utf-8-sig', index=False)

        # 3. 기회지역 CSV 저장
        if hasattr(analyzer, 'opportunities_df') and len(analyzer.opportunities_df) > 0:
            opp_filename = f'입지기회지역_{timestamp}.csv'
            analyzer.opportunities_df.to_csv(opp_filename, encoding='utf-8-sig', index=False)
            print(f"\n📁 파일 저장 완료:")
            print(f"   - 보고서: {txt_filename}")
            print(f"   - 경쟁업체 데이터: {csv_filename}")
            print(f"   - 기회지역 데이터: {opp_filename}")
        else:
            print(f"\n📁 파일 저장 완료:")
            print(f"   - 보고서: {txt_filename}")
            print(f"   - 경쟁업체 데이터: {csv_filename}")

        return txt_filename, csv_filename

    return None

def create_excel_report(analyzer):
    """Excel 형태의 상세 보고서 생성"""
    try:
        import openpyxl
        from openpyxl.styles import Font, PatternFill, Alignment
        from openpyxl.utils.dataframe import dataframe_to_rows
    except ImportError:
        print("Excel 보고서 생성을 위해 openpyxl을 설치하세요: !pip install openpyxl")
        return None

    from datetime import datetime
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f'경쟁분석_종합보고서_{timestamp}.xlsx'

    # 워크북 생성
    wb = openpyxl.Workbook()

    # 1. 요약 시트
    ws1 = wb.active
    ws1.title = "분석요약"

    # 헤더 스타일
    header_fill = PatternFill(start_color="366092", end_color="366092", fill_type="solid")
    header_font = Font(color="FFFFFF", bold=True)

    # 요약 정보 입력
    ws1['A1'] = "코딩학원 경쟁 분석 보고서"
    ws1['A1'].font = Font(size=16, bold=True)
    ws1['A3'] = f"생성일시: {datetime.now().strftime('%Y년 %m월 %d일 %H시 %M분')}"

    # 주요 지표
    ws1['A5'] = "주요 지표"
    ws1['A5'].font = header_font
    ws1['A5'].fill = header_fill

    df = analyzer.surroundings_df
    ws1['A6'] = f"총 경쟁업체 수: {len(analyzer.competitors)}개"
    ws1['A7'] = f"평균 주변 초등학교: {df['nearby_elementary'].mean():.1f}개, 학생 수 {df.['']}"
    ws1['A8'] = f"평균 주변 중학교: {df['nearby_middle'].mean():.1f}개"
    ws1['A9'] = f"평균 주변 상생업체: {df['total_supporting'].mean():.1f}개"

    if hasattr(analyzer, 'opportunities_df'):
        ws1['A10'] = f"발굴된 기회지역: {len(analyzer.opportunities_df)}개"

    # 2. 경쟁업체 데이터 시트
    ws2 = wb.create_sheet("경쟁업체_상세데이터")

    for r in dataframe_to_rows(analyzer.surroundings_df, index=False, header=True):
        ws2.append(r)

    # 헤더 스타일 적용
    for cell in ws2[1]:
        cell.font = header_font
        cell.fill = header_fill
        cell.alignment = Alignment(horizontal="center")

    # 3. 입지 기회 시트
    if hasattr(analyzer, 'opportunities_df') and len(analyzer.opportunities_df) > 0:
        ws3 = wb.create_sheet("입지기회_지역")

        for r in dataframe_to_rows(analyzer.opportunities_df, index=False, header=True):
            ws3.append(r)

        for cell in ws3[1]:
            cell.font = header_font
            cell.fill = header_fill
            cell.alignment = Alignment(horizontal="center")

    # 4. 구별 통계 시트
    ws4 = wb.create_sheet("구별_통계")

    # 경쟁업체 구별 분포
    district_stats = analyzer.competitors['district'].value_counts().reset_index()
    district_stats.columns = ['구', '경쟁업체_수']

    # 기회지역 구별 분포 추가
    if hasattr(analyzer, 'opportunities_df') and len(analyzer.opportunities_df) > 0:
        opp_stats = analyzer.opportunities_df['district'].value_counts().reset_index()
        opp_stats.columns = ['구', '기회지역_수']
        district_stats = district_stats.merge(opp_stats, on='구', how='left')
        district_stats['기회지역_수'] = district_stats['기회지역_수'].fillna(0)

    for r in dataframe_to_rows(district_stats, index=False, header=True):
        ws4.append(r)

    for cell in ws4[1]:
        cell.font = header_font
        cell.fill = header_fill
        cell.alignment = Alignment(horizontal="center")

    # 파일 저장
    wb.save(filename)
    print(f"📊 Excel 보고서 저장: {filename}")

    return filename

def download_all_files():
    """모든 생성된 파일을 한번에 다운로드"""
    try:
        from google.colab import files
        import glob

        # 생성된 모든 파일 찾기
        report_files = glob.glob("경쟁분석*.txt")
        csv_files = glob.glob("경쟁업체*.csv") + glob.glob("입지기회*.csv")
        excel_files = glob.glob("경쟁분석*.xlsx")
        map_files = glob.glob("*map.html")

        all_files = report_files + csv_files + excel_files + map_files

        print("📁 다운로드 가능한 파일들:")
        for file in all_files:
            print(f"   - {file}")

        # 파일 다운로드
        for file in all_files:
            try:
                files.download(file)
                print(f"✅ {file} 다운로드 완료")
            except Exception as e:
                print(f"❌ {file} 다운로드 실패: {e}")

    except ImportError:
        print("Google Colab 환경이 아닙니다. 로컬에서 파일을 확인하세요.")

#### 4. 실행

In [ ]:
if __name__ == "__main__":
    # 전체 분석 실행
    analyzer = run_complete_analysis()

    # 텍스트 보고서 생성 및 저장
    generate_detailed_report(analyzer, save_files=True)

    # Excel 보고서 생성 (선택사항)
    try:
        create_excel_report(analyzer)
    except Exception as e:
        print(f"Excel 보고서 생성 실패: {e}")

    # 모든 파일 다운로드 (Colab용)
    download_all_files()

학교 데이터: 992개
학원 데이터: 24295개
서울특별시 초중학교: 992개
서울특별시 학원: 24295개
경쟁업체: 252개
상생업체: 8365개
기타: 15678개

🔴 경쟁업체 샘플:
   - 서울게임교습소 (게임)
   - 에이블디코딩컴퓨터교습소 (nan)
   - 전자박사로봇교실로봇교습소 (로봇)

🟢 상생업체 샘플:
   - GLMA글로벌인생만남논술교습소 (보습·논술)
   - J바이올린음악교습소 (음악)
   - SCMA음악교습소 (음악)

🔍 경쟁업체 주변 환경 분석 중...
🏢 경쟁업체 주변 환경 분석 결과

📍 분석 대상: 252개 경쟁업체 (반경 1km 기준)

🏫 주변 학교 현황:
   • 평균 초등학교: 4.3개, 학생 수 3073.1746031746034명
   • 평균 중학교: 2.9개, 학생 수 1796.7539682539682명
   • 평균 총 학교 수: 7.2개

🎓 주변 상생업체 현황:
   • 평균 음악학원: 33.7개
   • 평균 태권도학원: 11.6개
   • 평균 논술학원: 68.2개
   • 평균 바둑/주산학원: 0.9개
   • 평균 총 상생업체: 114.6개

✅ 최적 입지 조건 (50%):
   • 초등학교 4개 이상, 초등학생 수 2910명 이상
   • 중학교 3개 이상, 중학생 수 1520명 이상
   • 상생업체 70개 이상

🎯 입지 기회 발굴 중...

🎯 발굴된 입지 기회: 41개 지역

📊 기회 지역 분포:
   • 성북구: 10개
   • 송파구: 5개
   • 양천구: 4개
   • 강동구: 4개
   • 강북구: 4개
   • 강남구: 3개
   • 도봉구: 2개
   • 영등포구: 2개
   • 동작구: 2개
   • 강서구: 1개

🗺️ 종합 분석 지도 생성 중...

✅ 분석 완료!
📁 파일 저장: competitor_analysis_map.html
📊 코딩학원 경쟁 분석 보고서
📅 생성일시: 2025년 07월 17일 10시 30분

1️⃣ 경쟁업체 현황
   총 252개 경쟁업체

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ 경쟁분석보고서_20250717_103029.txt 다운로드 완료


<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

✅ 입지기회지역_20250717_103029.csv 다운로드 완료


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ competitor_analysis_map.html 다운로드 완료


#### 원리   
1. 해당 키워드를 가진 업체를 경쟁 업체로 선정해서 경쟁업체 지도 만들기   
2. 해당 경쟁업체들 주변에 보통 학교 및 다른 학원이 무엇이 있는지, 주변이 어떻게 구성되어 있는지 분석+지도화 (ex. 경쟁업체 주변에는 평균적으로 초등학교 2개와 영어학원, 피아노학원, 태권도학원이 있다. 이런 식으로 결론이 나와야함)   
3. 2번같은 환경이 있지만 아직 경쟁업체(컴퓨터, 코딩학원)이 들어서지 않은 입지 추출   


---


🎯 시스템 기능 개요
1단계: 경쟁업체 지도 생성 🔴

코딩/컴퓨터 학원들을 경쟁업체로 분류
빨간색 마커로 지도에 표시==
경쟁업체 현황 파악

2단계: 주변 환경 분석 📊

각 경쟁업체 반경 1km 내 환경 분석
자동 계산:

주변 초등학교/중학교 개수
주변 상생업체 (영어, 수학, 피아노, 태권도, 논술학원) 개수


결론 도출: "경쟁업체 주변에는 평균적으로 초등학교 X개, 중학교 Y개, 영어학원 Z개가 있다"

3단계: 입지 기회 발굴 ⭐   

2단계에서 도출한 최적 환경 조건을 기준으로
아직 경쟁업체가 없거나 적은 지역 중에서
최적 조건을 만족하는 입지 기회 자동 추출
골드 스타 마커로 표시   

---


🗺️ 지도 레이어 구성

🏫 초중학교: 파란색/초록색 원형 마커   
🔴 경쟁업체: 빨간색 노트북 아이콘   
🟢 상생업체: 작은 색상별 원형 마커 (샘플)   
⭐ 입지 기회: 골드 스타 아이콘      

---


📊 추가 분석 기능
코드에는 다음과 같은 고급 분석 기능도 포함되어 있습니다:   

구별 경쟁업체 밀도 분석   
최적 입지 조건 자동 계산 (하위 25% 기준)   
거리 기반 환경 분석 (geopy 활용x 더 빠른 라이브러리씀)   
상생업체 세부 분류 (피아노, 논술 등)   
기회 지역 우선순위 랭킹     


역 기준 안할거임 어차피 초등학생이면 역보다 초등학교&주거지 인근이 훨씬 접근성이 좋을 것 같음.

태권도 포함함

^.^ 파이팅